### Парсинг HTML. BeautifulSoup, MongoDB

#### Задание 1

In [1]:
from bs4 import BeautifulSoup as bs
import requests as rq
from pprint import pprint
import pandas as pd

In [2]:
main_link = 'https://spb.hh.ru'

In [3]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.109 Safari/537.36',
          'Accept':'*/*'}

In [4]:
params = {'clusters':True,
          'enable_snippets':True,
          'text':'Data scientist',
          'L_save_area':True,
          'area':1,
          'from':'cluster_area',
          'showClusters':True}

In [5]:
response = rq.get(main_link + '/search/vacancy', headers=header, params=params).text
soup = bs(response, 'lxml')
vacancy_block = soup.find('div',{'class':'vacancy-serp'})
vacancy_list = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})

In [6]:
pages = []
while True:
    if soup.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}):
        next_page = main_link + soup.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
    else:
        break
    pages.append(next_page)
    response = rq.get(next_page, headers=header).text
    soup = bs(response, 'lxml')
    vacancy_block_ = soup.find('div',{'class':'vacancy-serp'})
    vacancy_list_ = vacancy_block_.find_all('div',{'class':'vacancy-serp-item'})
pprint(pages)

['https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=Data+scientist&page=1',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=Data+scientist&page=2',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=Data+scientist&page=3']


In [7]:
jobs = []

In [8]:
for job in vacancy_list + vacancy_list_:
    job_data = {}
    job_name = job.find('div',{'class':'vacancy-serp-item__info'}).getText()
    job_company = job.find('a',{'class':'bloko-link bloko-link_secondary'}).getText()
    job_city = job.find('span',{'class':'vacancy-serp-item__meta-info'}).getText().split()[0].replace(',', '')
    job_link = job.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
    job_data['name'] = job_name
    job_data['company'] = job_company
    job_data['city'] = job_city
    job_data['link'] = job_link
    jobs.append(job_data)

In [9]:
pd.DataFrame(jobs)

,name,company,city,link
0,Data Scientist,ООО КРИПТОН,Москва,https://spb.hh.ru/vacancy/37635578?query=Data%...
1,Python Data Scientist,Web Interactive World,Москва,https://spb.hh.ru/vacancy/37521387?query=Data%...
2,Senior Data Scientist,Gradient,Москва,https://spb.hh.ru/vacancy/37294434?query=Data%...
3,Data Scientist,ООО Варити,Москва,https://spb.hh.ru/vacancy/37569615?query=Data%...
4,Data Scientist (Computer Vision),СофтТелематика,Москва,https://spb.hh.ru/vacancy/37565065?query=Data%...
...,...,...,...,...
76,Pricing development manager,British American Tobacco Russia,Москва,https://spb.hh.ru/vacancy/37331973?query=Data%...
77,Ведущий аналитик данных/Data scientist,Сбербанк для экспертов,Москва,https://spb.hh.ru/vacancy/37302886?query=Data%...
78,Аналитик данных / Data scientist,"Программный Продукт, ИТ-компания",Москва,https://spb.hh.ru/vacancy/36953584?query=Data%...
79,Руководитель направления по аналитике данных,Сбербанк для экспертов,Москва,https://spb.hh.ru/vacancy/37303929?query=Data%...
